In [15]:
%matplotlib ipympl
import csv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import ipywidgets as widgets

filename=('DJI_0319.JPG')

img= mpimg.imread(filename)    
#start building the plot
fig, ax  = plt.subplots(figsize=[8,10])
#ax = fig.add_subplot(111)
ax.set_aspect('equal')

# Show the image
plt.imshow(img, cmap='gray' )

class DraggablePoint:
    lock = None #only one can be animated at a time
    def __init__(self, point):
        self.point = point
        
        self.press = None
        self.background = None

    def load(self):
        a=(QFileDialog.getOpenFileName())
        
    def connect(self):
        'connect to all the events we need'
        self.cidpress = self.point.figure.canvas.mpl_connect('button_press_event', self.on_press)
        self.cidrelease = self.point.figure.canvas.mpl_connect('button_release_event', self.on_release)
        self.cidmotion = self.point.figure.canvas.mpl_connect('motion_notify_event', self.on_motion)
        self.cidhover = self.point.figure.canvas.mpl_connect('key_press_event',self.on_pick)

    def on_pick(self,event):
        with out:
            print('you pressed', event.key, event.xdata, event.ydata)
        if event.key=='n': # position nose
            drs[0].point.center=event.xdata, event.ydata
        if event.key=='m': # position fin
            drs[1].point.center=event.xdata, event.ydata
        if event.key=='t': # position notch-tail
            drs[2].point.center=event.xdata, event.ydata
        self.point.figure.canvas.draw()
        
        if event.inaxes != self.point.axes: return
        if DraggablePoint.lock is not None: return
        contains, attrd = self.point.contains(event)
        if not contains: return
    
        larger_keys=['up','x']
        smaller_keys=['down','z']
    
        #print('you pressed', event.key, event.xdata, event.ydata, self.point.center[0], self.point.center[1])
        if event.key in larger_keys: 
            self.point.radius+=1
        if event.key in smaller_keys: 
            self.point.radius-=1
            if self.point.radius<5:
                self.point.radius=5
                         
        #calculatelength()                
        # redraw the full figure
        self.point.figure.canvas.draw()
        
        
    def on_press(self, event):
        'check if the mouse is within the circle'
        if event.inaxes != self.point.axes: return
        if DraggablePoint.lock is not None: return
        contains, attrd = self.point.contains(event)
        if not contains: return
        self.press = (self.point.center), event.xdata, event.ydata
        DraggablePoint.lock = self

        # draw everything but the selected rectangle and store the pixel buffer
        canvas = self.point.figure.canvas
        axes = self.point.axes
        self.point.set_animated(True)
        canvas.draw()
        self.background = canvas.copy_from_bbox(self.point.axes.bbox)

        # now redraw just the rectangle
        axes.draw_artist(self.point)

        # and blit just the redrawn area
        canvas.blit(axes.bbox)
        self.point.figure.canvas.draw()

    def on_motion(self, event):
        if DraggablePoint.lock is not self:
            return
        if event.inaxes != self.point.axes: return
        self.point.center, xpress, ypress = self.press
        dx = event.xdata - xpress
        dy = event.ydata - ypress
        self.point.center = (self.point.center[0]+dx, self.point.center[1]+dy)
        
        canvas = self.point.figure.canvas
        axes = self.point.axes
        
        # restore the background region
        canvas.restore_region(self.background)

        # redraw just the current rectangle
        axes.draw_artist(self.point)

        # blit just the redrawn area
        canvas.blit(axes.bbox)

    def on_release(self, event):
        'on release of the mouse reset the data'
        if DraggablePoint.lock is not self:
            return

        self.press = None
        DraggablePoint.lock = None

        # turn off the rect animation property and reset the background
        self.point.set_animated(False)
        self.background = None
        #update the onscreen calculations
        calculatelength()
        # redraw the full figure
        self.point.figure.canvas.draw()
        
    def disconnect(self):
        'disconnect all the stored connection ids'
        self.point.figure.canvas.mpl_disconnect(self.cidpress)
        self.point.figure.canvas.mpl_disconnect(self.cidrelease)
        self.point.figure.canvas.mpl_disconnect(self.cidmotion)

circles = [patches.Circle((120,10), 10, ec='r', fc='none', alpha=1, lw=2, ls='solid', label='N' ),
           patches.Circle((120,30), 10, ec='white', fc='none', alpha=1, lw=2, ls='solid', label='M'),
           patches.Circle((120,50), 10, ec='lightgreen', fc='none', alpha=1, lw=2, ls='solid', label='T'),
           patches.Circle((120,70), 10, ec='yellow', fc='none', alpha=1, lw=2, ls='solid', label='S'),
           patches.Circle((120,90), 10, ec='yellow', fc='none', alpha=1, lw=2, ls='solid', label='S')
           ]
#setup initial draggable points list
drs = []       
for circ in circles:
      ax.add_patch(circ)
      dr = DraggablePoint(circ)
      dr.connect()
      drs.append(dr)
    
filecsv=filename.split('.')[0]+ '.csv'

#check if a file with markers was saved before
try:
 with open(filecsv) as csv_file:
    csv_reader = csv.reader(csv_file)
    #reader = csv.reader(infile)
    a=next(csv_reader, None)  # skip the headers
    print(a)
    xmin=10000
    xmax=-1
    ymin=10000
    ymax=-1
    count=0
    for circ in circles: 
        csvdata=next(csv_reader,None)
        csv_x=float(csvdata[0])
        csv_y=float(csvdata[1])
        if count<3:
            circ.center=float(csv_x), float(csv_y)
            try:
              circ.radius=float(csvdata[2])  
            except:
              pass
            try:
              circ.set_label(csvdata[3])
            except:
              pass
            if count<3: #only use first 3 circles (Nose, Middle, Tail) for image dimensions        
                if float(csv_x)>xmax:
                    xmax=csv_x
                if float(csv_y)>ymax:
                    ymax=csv_y
                if float(csv_x)<xmin:
                    xmin=csv_x
                if float(csv_y)<ymin:
                    ymin=csv_y
        count+=1
        
    #zoom to area around nose/middle/tail markers           
    plt.axis([xmin*0.9,xmax*1.1,ymax*1.1,ymin*0.9])
except:
 print('no markers found')
 ax.set_xlabel('set markers, press N(nose), M(middle) or T(tail) at position')
   
leg = ax.legend(framealpha=0.2, facecolor='white')
plt.setp(leg.get_texts(), color='w')

#plt.show()
ResetZoomButton = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reset zoom',
    icon='arrows-alt'
)

def resetZoom(new):
    # Reset the x and y axes on the figure
    fig.axes[0].scale.min = None
    fig.axes[1].scale.min = None
    fig.axes[0].scale.max = None
    fig.axes[1].scale.max = None  

out= widgets.Output(layout={'border': '1px solid black'})
with out:
    plt.show()
    
ResetZoomButton.on_click(resetZoom)
widgets.VBox([out,ResetZoomButton], align_self='stretch')

FigureCanvasNbAgg()

['x', 'y', 'radius', 'label']
